In [30]:
import os
import sys

from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

src_dir = os.environ.get('srcdir')
print(src_dir)

sys.path.append(src_dir)

# always reload modules marked with "%aimport"
%load_ext autoreload
%autoreload 1

%aimport features.tidal_analysis
import features.tidal_analysis as ta

/home/ubuntu/bigStick/tidal_melting/src
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import xarray as xr
import os

atg_path = os.path.join(os.pardir,'data','analysis','external','atg','ATG_ocean_height_2010_0908.mat')
his_hly = os.path.join(os.environ.get('rawdir'),'waom10_full_forcing','ocean_his_hourly.nc')
grid = os.path.join(os.pardir,'data','analysis','raw','waom10_full_forcing','Data','waom10','waom10_grd_large.nc')

zeta = xr.open_dataset(his_hly).zeta

station_id = [24,30,1]
const_list = ['M2','O1']
stime=datetime.datetime(2007,1,1)

compare_tide(atg_path,zeta,zeta)

 Station:  20100- Davis
Amp(amp_err)[m]:  atg      roms  || phase(phase_err)[deg]:  atg       roms
M2 :             0.20   0.88(0.10)                       208.51   3.44(6.40)
O1 :             0.29   0.21(0.03)                       269.02   290.26(8.64)
 Station:  c16  (gravity measurements on the RIS)
Amp(amp_err)[m]:  atg      roms  || phase(phase_err)[deg]:  atg       roms
M2 :             0.03   0.44(0.09)                       311.51   204.32(12.04)
O1 :             0.26   0.41(0.04)                       190.82   171.18(5.82)


In [17]:
%%writefile ../src/features/compare_tide.py
from scipy.spatial import KDTree
import ttide as tt
import datetime
import numpy as np
import scipy.io as sio

def read_atg(atg_data,site_id,constit_list):

    site_data = {}
    for key in ['site_id','name','lat','lon','amp','Gphase','reclen','delta_t','meas_type','ref']:
        site_data[key] =np.squeeze(atg_data[key][0,0][site_id-1])
    site_data['constit']=np.squeeze(atg_data['constit'][0,0][:])
    site_data['name'] = site_data['name'].strip()

    cm2m = 1.0/100.0
    for const in constit_list:

        atg_con_ind = list(site_data['constit']).index(const)
        site_data[const]=np.array([site_data['amp'][atg_con_ind]*cm2m, site_data['Gphase'][atg_con_ind]])
        
    return site_data

def station_ttide(zeta_da,lat_t,lon_t,stime,constit_list):
    
    zeta_flat = zeta_da.stack(etaxi = ('eta_rho','xi_rho'))
    points = np.column_stack((zeta_flat.lat_rho.values,zeta_flat.lon_rho.values))
    tree = KDTree(points)
    
    target = np.column_stack((lat_t,lon_t))
    dist, ind = tree.query(target)
    
    tmp={}
    tmp['roms_signal'] = zeta_flat[:,ind].squeeze().values
    tmp['roms_ind'],tmp['dist_to ATG'] = ind,dist
    
    tmp['t_tide']=tt.t_tide(tmp['roms_signal'],dt=1,stime=stime,lat=zeta_flat.lat_rho[ind].values[0],out_style=None)


    for const in constit_list:
        tide_con_ind = list(tmp['t_tide']['nameu']).index(str.encode(const+'  ')) 
        tmp[const]=tmp['t_tide']['tidecon'][tide_con_ind]
        
    return tmp

def print_comparison(ttide_dict,atg_dict,constit_list):
      
        print(' Station: ',atg_dict['name'])
        print("Amp(amp_err)[m]:  atg      roms  || phase(phase_err)[deg]:  atg       roms")
        for con in constit_list:
            print(con,":             %0.2f"%atg_dict[con][0], "  %0.2f(%0.2f)"%(ttide_dict[con][0],ttide_dict[con][1]),\
                 "                      %0.2f"%atg_dict[con][1], "  %0.2f(%0.2f)"%(ttide_dict[con][2],ttide_dict[con][3])) 
            
def compare_tide(atg_mat_path,roms_zeta_da,stime=datetime.datetime(2007,1,1),constit_list = ['M2','O1'],station_list= [24,30]):

    mat_content = sio.loadmat(atg_mat_path)
    atg_data = mat_content['atg']
    
    for station in station_list:

        atg_dict = read_atg(atg_data,station,constit_list)
        lat = atg_dict['lat']
        lon = atg_dict['lon']
        ttide_dict = station_ttide(roms_zeta_da,lat,lon,stime,constit_list)

        print_comparison(ttide_dict,atg_dict,constit_list)

Overwriting ../src/features/compare_tide.py


In [47]:
import xarray as xr
from scipy.spatial import KDTree
import ttide as tt

roms_ds = xr.open_dataset(his_hly)

zeta_flat = roms_ds.zeta.stack(etaxi = ('eta_rho','xi_rho'))
points = np.column_stack((zeta_flat.lat_rho.values,zeta_flat.lon_rho.values))
tree = KDTree(points)

In [53]:
tmp = tide_comparison['24']
lat_t = tmp['atg']['lat']
lon_t = tmp['atg']['lon']
target = np.column_stack((lat_t,lon_t))
dist, ind = tree.query(target)
tmp['roms_signal'] = zeta_flat[:,ind].squeeze().values
tmp['roms_ind'],temp['dist_to ATG'] = ind,dist

tmp['t_tide']=tt.t_tide(tmp['roms_signal'],dt=1,stime=stime,lat=zeta_flat.lat_rho[ind].values[0],out_style=None)


In [54]:
tmp['t_tide']

{'dt': 1,
 'dz0': 0,
 'fu': array([ 0.00282193,  0.03873065,  0.04178075,  0.0805114 ,  0.08333333,
         0.1207671 ,  0.12511408,  0.1610228 ,  0.16384473,  0.16666667,
         0.20280355,  0.20844741,  0.2415342 ,  0.24435613,  0.24717807,
         0.28331495,  0.3220456 ]),
 'isComplex': False,
 'lat': -68.511783754802309,
 'ltype': 'nodal',
 'nameu': array([b'MSF ', b'O1  ', b'K1  ', b'M2  ', b'S2  ', b'M3  ', b'SK3 ',
        b'M4  ', b'MS4 ', b'S4  ', b'2MK5', b'2SK5', b'M6  ', b'2MS6',
        b'2SM6', b'3MK7', b'M8  '],
       dtype='|S4'),
 'ngood': 365,
 'nobs': 365,
 'nodcor': 'Greenwich phase computed with nodal\n                   corrections applied to amplitude\n                   and phase relative to center time\n',
 'ray': 1,
 'snr': array([   1.60397259,   45.62633091,   50.48112678,  101.17919337,
          67.37053   ,    2.61436361,    0.05228297,    7.4316941 ,
           0.90009454,    0.12063228,    0.63317532,    0.77316601,
           2.49747288,    9.702

In [66]:
str.encode(const+'   ')

b'O   '

In [79]:
for const in const_list:
    tmp[const]={}
    tide_con_ind = list(tmp['t_tide']['nameu']).index(str.encode(const+'  ')) 
    tmp[const]['t_tide']=tmp['t_tide']['tidecon'][tide_con_ind]

In [84]:
tmp['M2']

{'t_tide': array([ 0.87800499,  0.08728736,  3.43613934,  6.56165388])}

In [96]:
cm2m = 1.0/100.0
for const in const_list:
    atg_con_ind = list(tmp['atg']['constit']).index(const)
    tmp[const]['atg']=np.array([tmp['atg']['amp'][atg_con_ind]*cm2m, tmp['atg']['Gphase'][atg_con_ind]])

In [97]:
tmp['M2']

{'atg': array([   0.20099,  208.511  ]),
 't_tide': array([ 0.87800499,  0.08728736,  3.43613934,  6.56165388])}

In [123]:
def print_comparison(ttide_dict,atg_dict,site_id)
    for station in station_id:
        tmp = tide_comparison[str(station)]
        print(' Station: ',tmp['atg']['name'])
        print("Amp(amp_err)[m]:  atg      roms || phase(phase_err)[deg]:  atg       roms")
        for con in const_list:
            print(con,":             %0.2f"%tmp[con]['atg'][0], "  %0.2f(%0.2f)"%(tmp[con]['t_tide'][0],tmp[con]['t_tide'][1]),\
                 "                      %0.2f"%tmp[con]['atg'][1], "  %0.2f(%0.2f)"%(tmp[con]['t_tide'][2],tmp[con]['t_tide'][3])) 

 Station:  20100- Davis
Amp(amp_err)[m]:  atg      roms || phase(phase_err)[deg]:  atg       roms
M2 :             0.20   0.88(0.09)                       208.51   3.44(6.56)
O1 :             0.29   0.21(0.03)                       269.02   290.26(9.06)


In [101]:
%debug

> <ipython-input-100-6c372821f952>(6)<module>()
      2     tmp = tide_comparison[str(station)]
      3     print(' Station: ',tmp['atg']['name'])
      4     print("Amp/ampl_err [m]:  atg   roms || Gphase/phase_err [deg]:  atg    roms")
      5     for con in const_list:
----> 6         print(con,":     %0.2f"%tmp[con]['atg'][0], " %0.2f"%tmp[con]['t_tide'][0,1],             "           %0.2f"%tmp[con]['atg'][1], " %0.2f"%tmp[con]['t_tide'][2,3])

ipdb> tmp['M2']['atg'][0]
0.20099
ipdb> tmp[con]['t_tide'][0,1]
*** IndexError: too many indices for array
ipdb> tmp[con]['t_tide']
array([ 0.87800499,  0.08728736,  3.43613934,  6.56165388])
ipdb> tmp[con]['t_tide'][0]
0.87800499100246732
ipdb> q
